## **3. Fine-tuning modelu Bielik**

### **Uwaga!**
Ze względu na poufność danych, surowe raporty i adnotacje ekspertów nie są zawarte w tym repozytorium.

### **Model bazowy:** speakleash/Bielik-1.5B-v3 (https://huggingface.co/speakleash/Bielik-1.5B-v3)

#### **Import Bibliotek**

In [1]:
import json
import os
from pathlib import Path

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import plotly.express as px
from datasets import load_from_disk, Dataset
from sklearn.metrics import f1_score, accuracy_score, jaccard_score, hamming_loss
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    default_data_collator,
    EvalPrediction,
)

#### **3.1. Definicja parametrów treningu oraz stałych.**

In [2]:
TOKENIZED_DATA_PATH = "data/data_tokenized"
MODEL_OUTPUT_PATH = "models/Bielik-1.5B-v3-ESG"
MODEL_NAME = "speakleash/Bielik-1.5B-v3"

CRITERIA_NAMES = [
    'c1_transition_plan',
    'c2_risk_management',
    'c4_boundaries',
    'c6_historical_data',
    'c7_intensity_metrics',
    'c8_targets_credibility',
]
NUM_LABELS = len(CRITERIA_NAMES)

TRAINING_ARGS = {
    "per_device_train_batch_size": 1,      
    "per_device_eval_batch_size": 1,       
    "gradient_accumulation_steps": 8,      # Zwiększa efektywny rozmiar partii do 8 (1x8), stabilizując trening.
    "num_train_epochs": 3,                 # Można zmienić dla eksperymentów
    "learning_rate": 2e-5,                 # Można zmienić dla eksperymentów
    "fp16": True,                          # Włącza trening w trybie mieszanej precyzji, co przyspiesza i oszczędza VRAM.
    "logging_steps": 50,                   
    "evaluation_strategy": "epoch",        # Ewaluacja modelu na zbiorze walidacyjnym po każdej pełnej epoce.
    "save_strategy": "epoch",              # Zapisuje checkpoint modelu po każdej epoce, spójnie z ewaluacją.
    "save_total_limit": 2,                 # Przechowuje tylko 2 najlepsze checkpointy, oszczędzając miejsce na dysku.
    "load_best_model_at_end": True,        # Po treningu automatycznie wczytuje najlepszy znaleziony model.
    "metric_for_best_model": "f1_macro",   # Wybiera najlepszy model na podstawie F1-score
    "report_to": "none",                   
    "warmup_steps": 200,                   # Stopniowo zwiększa learning rate przez 200 kroków, stabilizując początek treningu.
    "weight_decay": 0.01,                  # Dodaje lekką regularyzację (L2), aby zapobiegać przeuczeniu.
}

#### **3.2. Wczytanie tokenizowanego zbioru.**

In [4]:
tokenized_datasets = load_from_disk(TOKENIZED_DATA_PATH)
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 3475
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 563
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 683
    })
})


#### **3.3. Obliczanie wag klas.**

Ze względu na silne niezbalansowanie zbioru, obliczamy wagi dla każdej z klas. Pomoże to modelowi zwrócić większą uwagę na rzadziej występujące klasy pozytywne.

In [7]:
def calculate_class_weights(dataset: Dataset) -> torch.Tensor:
    labels = np.array(dataset['labels'])
    pos_counts = np.sum(labels, axis=0)
    total_samples = len(labels)
    
    weights = []
    for count in pos_counts:
        weight = total_samples / (2 * count + 1e-6) if count > 0 else 1.0
        weights.append(weight)
        
    print(f"Obliczone wagi klas: {[f'{w:.2f}' for w in weights]}")
    return torch.tensor(weights, dtype=torch.float)

class_weights = calculate_class_weights(tokenized_datasets['train'])

Obliczone wagi klas: ['0.70', '1.25', '0.96', '1.62', '1.13', '1.91']


#### **3.4. Wczytanie modelu i tokenizera.**

In [8]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS,
    problem_type="multi_label_classification"
)

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at speakleash/Bielik-1.5B-v3 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### **3.5. Niestandardowa funkcja straty.**

Focal Loss to ulepszona wersja Binary Cross-Entropy, która redukuje wagę łatwo klasyfikowanych przykładów, pozwalając modelowi skupić się na trudnych, często mniejszościowych przypadkach.

In [9]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, pos_weight=None):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.pos_weight = pos_weight
    
    def forward(self, inputs, targets):
        bce_loss = nn.functional.binary_cross_entropy_with_logits(
            inputs, targets, reduction='none', pos_weight=self.pos_weight
        )
        pt = torch.exp(-bce_loss)
        focal_loss = self.alpha * (1 - pt)**self.gamma * bce_loss
        return focal_loss.mean()

#### **3.6. Przygotowanie niestandardowej klasy Trainer z ważoną stratą.**

Tworzymy własną klasę Trainer, która będzie używać funkcji FocalLoss wraz z obliczonymi wagami klas.

In [ ]:
class ESGTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights.to(self.args.device) if class_weights is not None else None

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        
        loss_fct = FocalLoss(alpha=0.25, gamma=2.0, pos_weight=self.class_weights)
        loss = loss_fct(logits, labels.float())
        
        return (loss, outputs) if return_outputs else loss